# Markdown Source

In [1]:
%%writefile tests/demo.md
---
amsthm:
  plain:
  - Theorem: [Lemma, Corollary]
  - With Space*
  - Proposition: Conjecture
  - WithoutSpace
  definition:
  - Definition
  remark:
  - Case
  parentcounter: chapter
...

<div class="Theorem" info="within parenthesis">
plain theoremstyle here
</div>

<div class="Theorem" id="simplestEquation"}
Label and reference:

$$E=mc^2$$
</div>

From the \ref{simplestEquation}, we see that...

<div class="With Space" info="within parenthesis">
Environment name has a space, and is unnumbered.
</div>

<div class="Lemma" info="within parenthesis">
This one share counter with Theorem
</div>

<div class="Definition" info="within parenthesis">
definition theoremstyle here
</div>

<div class="Case" info="within parenthesis">
remark theoremstyle here
</div>

<div class="proof" info="Proof of the Main Theorem">
Predefined proof theoremstyle here
</div>


Overwriting tests/demo.md


## Shorter Markdown Source

## In native

In [2]:
!pandoc -s -t native tests/demo.md

Pandoc (Meta {unMeta = fromList [("amsthm",MetaMap (fromList [("definition",MetaList [MetaInlines [Str "Definition"]]),("parentcounter",MetaInlines [Str "chapter"]),("plain",MetaList [MetaMap (fromList [("Theorem",MetaList [MetaInlines [Str "Lemma"],MetaInlines [Str "Corollary"]])]),MetaInlines [Str "With",Space,Str "Space*"],MetaMap (fromList [("Proposition",MetaInlines [Str "Conjecture"])]),MetaInlines [Str "WithoutSpace"]]),("remark",MetaList [MetaInlines [Str "Case"]])]))]})
[Div ("",["Theorem"],[("info","within parenthesis")])
 [Para [Str "plain",Space,Str "theoremstyle",Space,Str "here"]]
,RawBlock (Format "html") "<div class=\"Theorem\" id=\"simplestEquation\"}\nLabel and reference:\n\n$$E=mc^2$$\n</div>\n"
,Para [Str "From",Space,Str "the",Space,RawInline (Format "tex") "\\ref{simplestEquation}",Str ",",Space,Str "we",Space,Str "see",Space,Str "that..."]
,Div ("",["With","Space"],[("info","within parenthesis")])
 [Para [Str "Environment",Space,Str "name",Space,Str "has",Space,S

## In JSON

In [3]:
!pandoc -s -t json tests/demo.md

{"blocks":[{"t":"Div","c":[["",["Theorem"],[["info","within parenthesis"]]],[{"t":"Para","c":[{"t":"Str","c":"plain"},{"t":"Space"},{"t":"Str","c":"theoremstyle"},{"t":"Space"},{"t":"Str","c":"here"}]}]]},{"t":"RawBlock","c":["html","<div class=\"Theorem\" id=\"simplestEquation\"}\nLabel and reference:\n\n$$E=mc^2$$\n</div>\n"]},{"t":"Para","c":[{"t":"Str","c":"From"},{"t":"Space"},{"t":"Str","c":"the"},{"t":"Space"},{"t":"RawInline","c":["tex","\\ref{simplestEquation}"]},{"t":"Str","c":","},{"t":"Space"},{"t":"Str","c":"we"},{"t":"Space"},{"t":"Str","c":"see"},{"t":"Space"},{"t":"Str","c":"that..."}]},{"t":"Div","c":[["",["With","Space"],[["info","within parenthesis"]]],[{"t":"Para","c":[{"t":"Str","c":"Environment"},{"t":"Space"},{"t":"Str","c":"name"},{"t":"Space"},{"t":"Str","c":"has"},{"t":"Space"},{"t":"Str","c":"a"},{"t":"Space"},{"t":"Str","c":"space,"},{"t":"Space"},{"t":"Str","c":"and"},{"t":"Space"},{"t":"Str","c":"is"},{"t":"Space"},{"t":"Str","c":"unnumbered."}]}]]},{"t":"

# panflute testing filter

In [4]:
import panflute as pf
print(pf.__version__)
# pf.Doc.get_metadata?

1.9.6


In [5]:
%%writefile amsthm.py
#!/usr/bin/env python3
"""
Pandoc filter using panflute
"""

from collections import OrderedDict, ChainMap
import panflute as pf
import json


def parse_metadata(eachStyle):
    """
    Input: eachStyle is the metadata under the key of amsthm styles, i.e. plain, definition, or remark
    Output:
    environments: set of all environments
    unnumbered: set of all unnumbered environments
    counter_dict: a dictionary with keys as the environments and the values as the shared counter
    counter: a dictionary with keys as the shared counter, and values as the counter (int), initialized as 0
    shared_environments: a dictionary with keys as the shared counter, and values as all other environments using this counter
    standalone_environments: a set of standalone environments (that do not share its counter)
    """
    # initialize
    environments = set()
    unnumbered = set()
    counter_dict = {}
    counter = {}
    shared_environments = {}
    standalone_environments = set()
    # parsing
    for environment in eachStyle:
        # check if it is a dict: hence numbered environments with shared counters
        if isinstance(environment, dict):
            for main, shared in environment.items():
                environments.add(main)
                counter_dict[main] = main
                counter[main] = 0
                # check if it is a string (1 item) or a list
                if isinstance(shared, str):
                    environments.add(shared)
                    counter_dict[shared] = main
                    shared_environments[main] = [shared]
                else:
                    environments.update(shared)
                    for i in shared:
                        counter_dict[i] = main
                    shared_environments[main] = shared
        # check if it is unnumbered
        elif str(environment)[-1] == '*':
            environment = str(environment)[0:-1]
            environments.add(environment)
            unnumbered.add(environment)
        # else it is a numbered environment with unshared counter
        else:
            environment = str(environment)
            environments.add(environment)
            counter_dict[environment] = environment
            counter[environment] = 0
            standalone_environments.add(environment)
    return environments, unnumbered, counter_dict, counter, shared_environments, standalone_environments


def get_metadata(doc):
    """
    Getting the metadata:

    - doc.environments: set of all environments for matching the div
    - doc.unnumbered: set of unnumbered environments for checking if counter is needed
    - doc.counter_dict: dict to lookup the shared counter for each environment
    - doc.counter: dict of shared counters
    - doc.shared_environments: a dictionary with keys as the shared counter, and values as all other environments using this counter
    - doc.standalone_environments: a set of standalone environments (that do not share its counter)
    - doc.style: dicts of plain, definition, remark
    - header level mapping to part/chapter/section @todo
    """
    amsthm_metadata = doc.get_metadata('amsthm')
    # parse each styles
    all_parsed_metadata = {}
    amsthm_style = ('plain', 'definition', 'remark')
    for i in amsthm_style:
        all_parsed_metadata[i] = parse_metadata(amsthm_metadata[i])
    # store output in doc
    # (0, 1, 2, 3, 4, 5) corresponds to (environments, unnumbered, counter_dict, counter, shared_environments, standalone_environments)
    doc.environments = set().union(*[all_parsed_metadata[i][0] for i in amsthm_style])
    doc.unnumbered = set().union(*[all_parsed_metadata[i][1] for i in amsthm_style])
    doc.counter_dict = dict(ChainMap(*[all_parsed_metadata[i][2] for i in amsthm_style]))
    doc.counter = dict(ChainMap(*[all_parsed_metadata[i][3] for i in amsthm_style]))
    doc.shared_environments = dict(ChainMap(*[all_parsed_metadata[i][4] for i in amsthm_style]))
    doc.standalone_environments = set().union(*[all_parsed_metadata[i][5] for i in amsthm_style])
    doc.style = {i: all_parsed_metadata[i][0] for i in amsthm_style}
    # get parent counter
    doc.parentcounter = amsthm_metadata['parentcounter']


def define_latex_enviroments(doc):
    """
    For LaTeX output only, convert the metadata obtained in get_metadata to LaTeX amsthm environment's definition
    """
    latex_amsthm_def = []
    amsthm_style = ('plain', 'definition', 'remark')
    for style in amsthm_style:
        if doc.style[style]:
            latex_amsthm_def += [r'\theoremstyle{' + style + '}']
            for i in doc.style[style]:
                # unnumbered environment
                if i in doc.unnumbered:
                    latex_amsthm_def += [r'\newtheorem*{' + i + '}{' + i + '}']
                # numbered, standalone environment
                elif i in doc.standalone_environments:
                    latex_amsthm_def += [r'\newtheorem{' + i + '}{' + i + '}[' + doc.parentcounter + ']']
                # numbered, shared environments
                elif i in doc.shared_environments:
                    latex_amsthm_def += [r'\newtheorem{' + i + '}{' + i + '}[' + doc.parentcounter + ']']
                    for shared in doc.shared_environments[i]:
                        latex_amsthm_def += [r'\newtheorem{' + shared + '}['+ i + ']{' + shared + '}']
    doc.content.insert(0, pf.RawBlock('\n'.join(latex_amsthm_def), format='latex'))

def prepare(doc):
    get_metadata(doc)
    if doc.format == 'latex':
        define_latex_enviroments(doc)


def amsthm(elem, doc):
    pass


def finalize(doc):
#     print('doc.environments:', doc.environments)
#     print('doc.unnumbered:', doc.unnumbered)
#     print('doc.counter_dict:', doc.counter_dict)
#     print('doc.counter:', doc.counter)
#     print('doc.shared_environments:', doc.shared_environments)
#     print('doc.style:', doc.style)
#     print('doc.parentcounter:', doc.parentcounter)
    pass


def main(doc=None):
    return pf.run_filter(amsthm,
                         prepare=prepare,
                         finalize=finalize,
                         doc=doc) 


if __name__ == '__main__':
    main()

Overwriting amsthm.py


In [6]:
!chmod +x amsthm.py

## Output

In [7]:
!pandoc -s -t json tests/demo.md | ./amsthm.py

{"pandoc-api-version":[1,17,0,4],"meta":{"amsthm":{"t":"MetaMap","c":{"remark":{"t":"MetaList","c":[{"t":"MetaInlines","c":[{"t":"Str","c":"Case"}]}]},"definition":{"t":"MetaList","c":[{"t":"MetaInlines","c":[{"t":"Str","c":"Definition"}]}]},"plain":{"t":"MetaList","c":[{"t":"MetaMap","c":{"Theorem":{"t":"MetaList","c":[{"t":"MetaInlines","c":[{"t":"Str","c":"Lemma"}]},{"t":"MetaInlines","c":[{"t":"Str","c":"Corollary"}]}]}}},{"t":"MetaInlines","c":[{"t":"Str","c":"With"},{"t":"Space"},{"t":"Str","c":"Space*"}]},{"t":"MetaMap","c":{"Proposition":{"t":"MetaInlines","c":[{"t":"Str","c":"Conjecture"}]}}},{"t":"MetaInlines","c":[{"t":"Str","c":"WithoutSpace"}]}]},"parentcounter":{"t":"MetaInlines","c":[{"t":"Str","c":"chapter"}]}}}},"blocks":[{"t":"Div","c":[["",["Theorem"],[["info","within parenthesis"]]],[{"t":"Para","c":[{"t":"Str","c":"plain"},{"t":"Space"},{"t":"Str","c":"theoremstyle"},{"t":"Space"},{"t":"Str","c":"here"}]}]]},{"t":"RawBlock","c":["html","<div class=\"Theorem\" id=\"

In [8]:
!pandoc -t latex tests/demo.md -F ./amsthm.py

\theoremstyle{plain}
\newtheorem*{With Space}{With Space}
\newtheorem{Proposition}{Proposition}[chapter]
\newtheorem{Conjecture}[Proposition]{Conjecture}
\newtheorem{WithoutSpace}{WithoutSpace}[chapter]
\newtheorem{Theorem}{Theorem}[chapter]
\newtheorem{Lemma}[Theorem]{Lemma}
\newtheorem{Corollary}[Theorem]{Corollary}
\theoremstyle{definition}
\newtheorem{Definition}{Definition}[chapter]
\theoremstyle{remark}
\newtheorem{Case}{Case}[chapter]

plain theoremstyle here

From the \ref{simplestEquation}, we see that\ldots{}

Environment name has a space, and is unnumbered.

This one share counter with Theorem

definition theoremstyle here

remark theoremstyle here

Predefined proof theoremstyle here


# YAML Test

In [9]:
YAML = """
amsthm:
  plain:
  - Theorem: Lemma
  - With Space*
  definition:
  - Definition
  remark:
  - Case
  parentcounter:
  - chapter
"""

In [10]:
import yaml

In [11]:
options = yaml.load(YAML)
print(options)

{'amsthm': {'parentcounter': ['chapter'], 'definition': ['Definition'], 'remark': ['Case'], 'plain': [{'Theorem': 'Lemma'}, 'With Space*']}}


In [12]:
options['amsthm']

{'definition': ['Definition'],
 'parentcounter': ['chapter'],
 'plain': [{'Theorem': 'Lemma'}, 'With Space*'],
 'remark': ['Case']}